## Imports

In [ ]:
import os
import openai
import nest_asyncio
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import chromadb
from chromadb import Client

## Initialisierung

In [ ]:
# Laden der Umgebungsvariablen
load_dotenv()

# OpenAI Client initialisieren
openAIclient = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

# ChromaDB Client initialisieren
chromaDBclient = Client()

# Event-Loop anpassen
nest_asyncio.apply()


## Dokumente aus PDF-Dateien laden und in ChromaDB speichern

In [ ]:
# Parser für PDF-Dateien definieren
parser = LlamaParse(result_type="text", language="de")
file_extractor = {".pdf": parser}

# Dokumente aus PDF-Dateien laden
documents = SimpleDirectoryReader(input_files=['testdokument.pdf'], file_extractor=file_extractor).load_data()
print("Dokumente geladen:", documents)

# ChromaDB-Client einrichten
collection = chromaDBclient.create_collection("documents")

## Keine Abschnitte, nur gesamtes Dokument speichern
for document in documents:
	content = document.text
	doc_id = document.get_doc_id()
	metadata = {"source": "testdokument.pdf"}
	collection.add(documents=[content], metadatas=[metadata], ids=[doc_id])

# Speichern der Dokumentabschnitte in ChromaDB
# for document in documents:
#     content = document.text
#     doc_id = document.get_doc_id()
#     sections = content.split("\n")

    
#     for i, section in enumerate(sections):
#         section_id = f"{doc_id}_section_{i}"
#         metadata = {"source": "testdokument.pdf", "section": i}
        
#         # Hinzufügen jedes Abschnitts zur Sammlung
#         collection.add(documents=[section], metadatas=[metadata], ids=[section_id])

print("Dokument erfolgreich in ChromaDB gespeichert.")

## Funktion zum Abfragen von ChromaDB

In [ ]:
def query_chromadb(query_text: str, n_results: int = 4):
    try:
        # Hier wird die ChromaDB-Abfrage ausgeführt
        collection = chromaDBclient.get_collection("documents_v2")  # Sicherstellen, dass der Collection-Name korrekt ist
        results = collection.query(query_texts=[query_text], n_results=n_results)
        print("Ergebnisse von ChromaDB:", results)
        return results
    except Exception as e:
        print(f"Fehler bei der Abfrage an ChromaDB: {e}")
        return None

## Funktion zum Abrufen einer Antwort von OpenAI

In [ ]:
def get_openai_response(user_query: str, context: str) -> str:
    # Definiere die Nachrichten im richtigen Format für die Chat-API
    messages = [
        {"role": "system", "content": "Du bist ein Assistent, der Fragen zum Thema Ernährung wissenschaftlich fundiert aufgrund von Kontext beantwortet. Beziehe dich  nur auf den Kontext, um die Fragen zu beantworten."},
        {"role": "user", "content": f"Hier ist ein Kontext:\n{context}\nFrage: {user_query}"}
    ]
    
    try:
        # Senden der Anfrage an OpenAI mit der richtigen Methode
        chat_completion = openAIclient.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,  # Nachricht an das Modell
            max_tokens=150,
            temperature=0.7
        )
        
        # Rückgabe der Antwort des Modells
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"Fehler bei der Anfrage an OpenAI: {e}")
        return "Fehler bei der Anfrage."

## Hauptprogramm

In [ ]:
# Beispielabfrage an ChromaDB
results = query_chromadb("Anfrage zu Vitaminen")

# Prüfung, ob Ergebnisse vorhanden sind und ob sie die erwartete Struktur haben
if results and isinstance(results, dict) and 'documents' in results:
    # Debugging-Ausgabe für die Dokumentstruktur
    print("Struktur von documents in results:", results['documents'])
    
    # Extraktion des Kontextes aus dem ersten Dokument
    context = "\n".join(results['documents'][0]) if results['documents'][0] else "Kein Kontext verfügbar"
    user_query = "Müssen Vitamine über die Nahrung aufgenommen werden?"
    
    # Anfrage an OpenAI mit dem extrahierten Kontext und der Benutzerfrage
    answer = get_openai_response(user_query, context)
    print("Antwort von OpenAI:", answer)
else:
    print("Keine Dokumente gefunden oder Ergebnisse haben ein unerwartetes Format.")